<a href="https://colab.research.google.com/github/basisvectors/Scene-Change-Detection/blob/main/Scene_Change_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scenedetect[opencv] --upgrade --quiet
!pip install ffmpeg --quiet
!pip install -U yt-dlp --quiet
!apt-get -qq install mkvtoolnix

!git clone https://github.com/slhck/bufferer.git
!pip install bufferer


     |████████████████████████████████| 98 kB 3.8 MB/s 
     |████████████████████████████████| 2.6 MB 5.5 MB/s 
     |████████████████████████████████| 357 kB 61.4 MB/s 
     |████████████████████████████████| 218 kB 61.7 MB/s 
     |████████████████████████████████| 2.0 MB 48.6 MB/s 
     |████████████████████████████████| 111 kB 69.1 MB/s 
Selecting previously unselected package libmagic-mgc.
(Reading database ... 156210 files and directories currently installed.)
Preparing to unpack .../0-libmagic-mgc_1%3a5.32-2ubuntu0.4_amd64.deb ...
Unpacking libmagic-mgc (1:5.32-2ubuntu0.4) ...
Selecting previously unselected package libmagic1:amd64.
Preparing to unpack .../1-libmagic1_1%3a5.32-2ubuntu0.4_amd64.deb ...
Unpacking libmagic1:amd64 (1:5.32-2ubuntu0.4) ...
Selecting previously unselected package libebml4v5:amd64.
Preparing to unpack .../2-libebml4v5_1.3.5-2ubuntu0.1_amd64.deb ...
Unpacking libebml4v5:amd64 (1.3.5-2ubuntu0.1) ...
Selecting previously unselected package libmatroska6v5:

In [ ]:
#@markdown #Video Downloder{ vertical-output: true }
# import os
import shutil
import os
# Standard PySceneDetect imports:
from scenedetect import VideoManager
from scenedetect import SceneManager

# For content-aware scene detection:
from scenedetect.detectors import ContentDetector

def find_scenes(video_path, threshold=30.0):
    # Create our video & scene managers, then add the detector.
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(
        ContentDetector(threshold=threshold))

    # Improve processing speed by downscaling before processing.
    video_manager.set_downscale_factor()

    # Start the video manager and perform the scene detection.
    video_manager.start()
    scene_manager.detect_scenes(frame_source=video_manager)

    # Each returned scene is a tuple of the (start, end) timecode.
    scene_list = scene_manager.get_scene_list()
    # print(len(scene_list))
    splits = ""
    for i in scene_list:
      splits += f"[{str(i[1].get_seconds())},{DURATION}],"
    splits = splits[:-1]

    return scene_list, splits

url = "https://www.youtube.com/watch?v=ZVjo86OEeFc" #@param {type:"string"}
id = "/content/" + url.split("=")[1]

!mkdir {id}

# shutil.copytree("./bufferer/spinners/",f"{id}/spinners/", copy_function=shutil.copy2)

# !cd {id}
# os.chdir(id)
# os.listdir()
filepath = f"{id}/input.mp4"
!yt-dlp {url} -o {filepath} -f mp4
#@markdown ### Scene change pause duration
DURATION = 1 #@param {type:"number"}

[youtube] ZVjo86OEeFc: Downloading webpage
[youtube] ZVjo86OEeFc: Downloading android player API JSON
[info] ZVjo86OEeFc: Downloading 1 format(s): 22
[download] Destination: /content/ZVjo86OEeFc/input.mp4
[download] 100% of 6.89MiB in 00:00


In [ ]:
scene_list, splits = find_scenes(filepath)
print(splits)

100%|██████████| 720/720 [00:05<00:00, 142.69frames/s]

[2.8361666666666663,1],[5.588916666666666,1],[6.5065,1],[8.133125,1],[9.759749999999999,1],[11.011,1],[12.429083333333333,1],[13.805458333333332,1],[15.014999999999999,1],[15.682333333333332,1],[17.100416666666664,1],[18.143124999999998,1],[18.935583333333334,1],[19.894875,1],[20.812458333333332,1],[22.230541666666664,1],[22.897875,1],[23.732041666666664,1],[26.276249999999997,1],[27.861166666666666,1],[29.195833333333333,1],[30.029999999999998,1]


In [ ]:
#@title Preview { vertical-output: true }

spinner = "./bufferer/spinners/spinner-256-white.png" #@param ["./bufferer/spinners/spinner-256-white.png", "./bufferer/spinners/spinner-256-black.png"]

!bufferer -i {filepath} -b {splits} -o output.mp4 -v libx264 -a aac --verbose -s {spinner}

from IPython.display import display
from ipywidgets import Video

Video.from_file("/content/output.mp4")

[info] running command for processing video
ffmpeg -nostdin -threads 1 -n -i /content/ZVjo86OEeFc/input.mp4 -filter_complex '[0:v]loop=loop=23:size=1:start=68,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=157,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=202,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=264,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=326,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=379,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=436,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=492,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=544,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=583,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=640,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=688,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=730,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=776,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=821,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:start=878,setpts=N/FRAME_RATE/TB,loop=loop=23:size=1:sta

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x87q\x18mdat\x00\x…